In [1]:
import dotenv
dotenv.load_dotenv(override=True)

import os
import uuid
from typing import List, Union
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
from pydantic_ai.messages import ToolReturnPart, TextPart, ModelResponse
from pydantic_ai.mcp import MCPServerStdio
import asyncio
import nest_asyncio
nest_asyncio.apply()

# Agent Settings

In [ ]:
# Initialize Openai-like model
model = OpenAIModel(
    'gemma3:27b-it-qat',
    provider=OpenAIProvider(
        base_url=os.getenv("AGENT_BASE_URL"), 
        api_key=os.getenv("AGENT_API_KEY")
    ),
)
# model = OpenAIModel(
#     'mistral-small3.1-24b-instruct-2503',
#     provider=OpenAIProvider(
#         base_url=os.getenv("AGENT_BASE_URL"), 
#         api_key=os.getenv("AGENT_API_KEY")
#     ),
# )

In [3]:
# structured output
from pydantic import BaseModel

class Reference(BaseModel):
    relevant: bool
    text: str

class RAGFormat(BaseModel):
    reference: List[Reference] # some model cannot use nested structure like this
    final_answer: str

In [4]:
# MCP server
rag_mcp_command = f"python3 {os.getcwd()}/1_rag_server.py stdio"
rag_mcp_server = MCPServerStdio(
    command=rag_mcp_command.split(" ")[0],
    args=rag_mcp_command.split(" ")[1:],
    env={
        "OPENAI_BASE_URL": os.getenv("OPENAI_BASE_URL"),
        "OPENAI_API_KEY": os.getenv("OPENAI_API_KEY"),
        },
)

# Agent initialization
seeker_agent = Agent(
    model=model,
    instructions=(
        "You are a helpful and intelligent assistant. The user you are helping speaks Traditional Chinese and comes from Taiwan, so in most cases, you should respond in Traditional Chinese. \n"
        "Behavior Rules: \n"
        "1. Direct Answering: If the question is clear and within your knowledge, answer directly.\n"
        "2. Retrieval-Based Answering: If the question requires specialized or external knowledge, retrieve relevant documents from the specific vector database. When retrieving from the database, the user's original intent should be preserved as much as possible, and the clarity of the question's meaning should be maintained.\n"
        "3. Clarification: If the question is vague or unclear, ask clarifying questions to understand the user’s intent before responding.\n"
        "4. If the response doesn't fit the RAGFormat, just reply in plain text."
    ),
    mcp_servers=[rag_mcp_server],
    output_type=Union[RAGFormat, str]
)

In [5]:
# Multi-turn conversation
def convert_tool_role_into_assistant(message_history):
    # check if the last message is ToolReturnPart
    if isinstance(message_history[-1].parts[0], ToolReturnPart):
        raw_assistant_message = message_history[-2]
        assistant_message = ModelResponse(
            parts=[
                TextPart(
                    content = raw_assistant_message.parts[0].args,
                    part_kind = "text",
                ),
            ],
            usage = raw_assistant_message.usage,
            model_name = raw_assistant_message.model_name,
            timestamp = raw_assistant_message.timestamp,
            kind = raw_assistant_message.kind,
            vendor_id = raw_assistant_message.vendor_id
            )
        return message_history + [assistant_message]
    else:
        return message_history

async def multi_turn_conversation(start_agent, full_history):
    conversation_id = uuid.uuid4().hex[:16]
    full_history[conversation_id] = []

    message_history = []
    while True:
        user_input = input("You: ")
        print(f"You: {user_input}")

        if user_input == "exit":
            print("Agent: Goodbye!")
            return result

        if message_history != []:
            result = await start_agent.run(user_prompt=user_input, message_history = message_history)
        else:
            result = await start_agent.run(user_prompt=user_input)
        
        print(f"Agent: {result.output}")

        message_history = convert_tool_role_into_assistant(result.all_messages())
        full_history[conversation_id] = message_history
    

In [6]:
full_history = {}
async def main(full_history):
    async with seeker_agent.run_mcp_servers():
        result = await multi_turn_conversation(seeker_agent, full_history)
    return result

In [36]:
result = asyncio.run(main(full_history))

You: 今天天氣真好～臺北現在33度
Agent: 是啊，今天臺北天氣確實不錯！33度是很舒服的溫度。請問有什麼我可以幫您的嗎？
You: 我想要請假出去玩，有沒有什麼限制？
Agent: reference=[Reference(relevant=True, text='此為依據勞動基準法所訂定的最基準工作時間。\n一、每週工作時數不得超過四十小時。\n二、每日工作時數為八小時。\n三、每日工作時間為\n08:30 開始上班\n12:00 午休及午餐時間\n13:00 下午工作時間開始\n15:00 下午休息時間\n15:15 繼續工作\n17:30 下班\n子女未滿一歲須員工親自哺乳者，除規定之休息時間外，本公司將每日另給哺乳時間二次，每次以三十分鐘為度，哺乳時間，視為工作時間。\n員工為撫育未滿三歲子女，得請求下列所定事項之一：\n1.\t每天減少工作時間一小時；減少之工作時間，不得請求報酬。\n2.\t調整工作時間。\n員工為前二項哺乳時間、減少或調整工時之請求時，本公司不得拒絕或視為缺勤而影響其全勤獎金、考績或為其他不利之處分。'), Reference(relevant=True, text='一、休假種類包含：特休假、生日假、普通傷病假、事假、家庭照顧假、婚假、喪假、產假、陪產假、產檢假、安胎假、公傷假、生理假、公假共十四種。\n二、請假方法：員工因故必須請假者，應事先填寫請假單或口頭敘明理由經核定後方可離開工作崗位或不出勤。\n \n請假天數，核准主管權限及事前天數辦法如下：\n請假天數\t事前天數\t核准主管權限\n0<天數≦5\t三天前\t部門主管\n5<天數≦10\t一週前\t處級主管\n天數>10\t兩週前\t業務/事業群總經理\n三、臨時狀況：同仁因突發事件或生病必須於當日請假者，得於當日10:00前以電話向主管報備，並於事後補辦請假手續。\n四、同仁未依規定辦理請假手續者，事後又不補辦請假手續並說明原因者，視同曠職。')] final_answer='請假規定依照勞動基準法，以及公司內部規定辦理。休假種類包含特休假、事假、婚假等等。請假天數不同，需要提前請假的時數也不同，通常部門主管可以核准5天以內的請假，超過5天需要處級主管核准，超過10天則需要總經理核准。 如果是臨時狀況，當天10點前向主管報備即可。請記得事先填寫請假單喔！'
Yo

In [37]:
result.all_messages()

[ModelRequest(parts=[UserPromptPart(content='今天天氣真好～臺北現在33度', timestamp=datetime.datetime(2025, 6, 18, 9, 17, 55, 946168, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], instructions="You are a helpful and intelligent assistant. The user you are helping speaks Traditional Chinese and comes from Taiwan, so in most cases, you should respond in Traditional Chinese. \nBehavior Rules: \n1. Direct Answering: If the question is clear and within your knowledge, answer directly.\n2. Retrieval-Based Answering: If the question requires specialized or external knowledge, retrieve relevant documents from the specific vector database. When retrieving from the database, the user's original intent should be preserved as much as possible, and the clarity of the question's meaning should be maintained.\n3. Clarification: If the question is vague or unclear, ask clarifying questions to understand the user’s intent before responding.", kind='request'),
 ModelResponse(parts=[TextPart(content='是啊，

In [40]:
result.new_messages()

[ModelRequest(parts=[UserPromptPart(content='我想要請特休假三天', timestamp=datetime.datetime(2025, 6, 18, 9, 18, 35, 214761, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], instructions="You are a helpful and intelligent assistant. The user you are helping speaks Traditional Chinese and comes from Taiwan, so in most cases, you should respond in Traditional Chinese. \nBehavior Rules: \n1. Direct Answering: If the question is clear and within your knowledge, answer directly.\n2. Retrieval-Based Answering: If the question requires specialized or external knowledge, retrieve relevant documents from the specific vector database. When retrieving from the database, the user's original intent should be preserved as much as possible, and the clarity of the question's meaning should be maintained.\n3. Clarification: If the question is vague or unclear, ask clarifying questions to understand the user’s intent before responding.", kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='fi

# Tools Setting

In [42]:
# 直接用decorator設定，不用再額外放入Agent()
@seeker_agent.tool_plain
def calculate_bmi(weight_kg: float, height_m: float) -> float:
    """Calculate BMI given weight in kg and height in meters"""
    return weight_kg / (height_m**2)

In [46]:
result = asyncio.run(main(full_history))

You: 我的身高160公分，體重45公斤
Agent: 您的BMI是17.58。根據BMI指數，您屬於體重過輕的範圍。建議您可以諮詢醫生或營養師，看看是否有需要改善的地方，並確保飲食均衡和適當的運動。
You: exit
Agent: Goodbye!
